In [2]:
import pandas as pd 
import numpy as np 
import scipy.stats as stats 

In [3]:
df1 = pd.read_csv('train.csv')
print(df1.shape)

(262, 21)


In [4]:
df1['Target'] = df1['class'].replace({'A':0 ,'B':1,'C':2})


In [125]:
X = pd.get_dummies(df1.drop(columns=['id','father','mother','gender','class','Target']))
#df1[['trait','SNP_01','SNP_02','SNP_08','SNP_11','SNP_13','SNP_14','SNP_15']]
Y = df1['Target']

In [94]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute   import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

In [126]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3)

In [127]:
pipe_list = [('impute', SimpleImputer()),
            ('scaler',StandardScaler()),
            ('model', SGDClassifier())]
pipe_model = Pipeline(pipe_list)
pipe_model

Pipeline(steps=[('impute', SimpleImputer()), ('scaler', StandardScaler()),
                ('model', SGDClassifier())])

In [139]:
hypeer_parameter = {'model__class_weight':['balanced'],
                    'model__loss':['log']}
#params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
grid_model = GridSearchCV(pipe_model, param_grid=hypeer_parameter, cv=5, 
                         n_jobs = -1 ,scoring='f1')
grid_model.fit(X_train, Y_train)

c:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(
c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'model__class_weight': ['balanced'],
                         'model__loss': ['log']},
             scoring='f1')

In [140]:
best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('impute', SimpleImputer()), ('scaler', StandardScaler()),
                ('model', SGDClassifier(class_weight='balanced', loss='log'))])

In [141]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred  = best_model.predict(X_test)

In [142]:
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        49
           1       1.00      0.99      0.99        83
           2       0.98      1.00      0.99        51

    accuracy                           0.99       183
   macro avg       0.99      1.00      0.99       183
weighted avg       0.99      0.99      0.99       183



In [147]:
print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       0.85      0.90      0.88        31
           2       0.92      0.82      0.87        28

    accuracy                           0.90        79
   macro avg       0.91      0.91      0.91        79
weighted avg       0.90      0.90      0.90        79



In [144]:
df2 = pd.read_csv('test.csv')
X1 = pd.get_dummies(df2.drop(columns=['id','father','mother','gender'
                                    ]))

In [145]:
X1_test_pred = best_model.predict(X1)

In [146]:
X1_test_pred = pd.DataFrame(X1_test_pred)
X1_test_pred= X1_test_pred.replace({0:'A',1:'B',2:'C'})

X1_test_pred.value_counts()


B    82
A    52
C    41
dtype: int64

In [195]:
X1_test_pred['id'] = df2['id']
X1_test_pred['class'] = X1_test_pred[0]

In [196]:
X1_test_pred = X1_test_pred.drop(columns=[0])
X1_test_pred = X1_test_pred.set_index('id')

In [197]:
X1_test_pred.to_csv('RF_rmSNP.csv',mode='w')